# 1. 数据准备模块

### 1.1 第三方库

In [1]:
import os
import numpy as np
import pandas as pd
from  PIL import Image
import matplotlib.pyplot as plt
from matplotlib import image
import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import sem
from sklearn import metrics

### 1.2 数据导入函数

In [2]:
#--------------------#
# 返回一个存放图片名的list
#--------------------#
def get_file_list(path = "/archive"):
    file_list = []
    for dirname,_, filenames in os.walk(path):
        for filename in filenames:
            file_list.append(os.path.join(dirname, filename))
    return file_list

#--------------------#
# 读取file_list对应的图片，resize后标准化，最后返回 n*256*256*3 的np.array 
# 这里DSNET数据集的图片本身就是256*256大小
#--------------------#
def get_imgs(file_list, img_resize=(256,256)):
    imgs = []
    for fname in file_list:
        img = Image.open(fname)
        img = img.resize(img_resize)
        img = img.convert('L')
        imgs.append(np.array(img) / 255)
        del img
    return np.array(imgs)


def get_target(file_list):
    # 0: 没有裂缝 1：有裂缝
    targets = []
    for fname in file_list:  
        if (fname.find('Non-cracked') != -1):
            targets.append(0)
        if (fname.find('Cracked') != -1):
            targets.append(1)
    return np.array(targets)

### 1.3 进一步封装数据导入函数

In [3]:
def get_X_y(file_list, channel):
    imgs = get_imgs(file_list)
    targets = get_target(file_list)
    
    if channel=="MachineLearning":   
        # 将图片扁平化
        imgs = np.reshape(imgs, (imgs.shape[0], -1))
    
    # 可以试一下转换成灰度图
    
    return imgs,targets

# 2. 机器学习模块

### 2.1 准备用于机器学习的数据

In [4]:
file_list = get_file_list("D:/Data/Apps/Python/Kaggle_SDNET/archive/all")
file_list = random.choices(file_list, k=len(file_list))
train_list = file_list[:int(len(file_list)*0.9)]
test_list = file_list[int(len(file_list)*0.1):]

### 2.2 定义评估函数

In [5]:
def evaluate_cross_validation(clf, X, y, K):
    # 创建 K-折交叉验证迭代器对象
    cv = KFold(K, shuffle=True, random_state=0)
    #cv = KFold(len(y), K, shuffle=True, random_state=0)
    # 计算返回分数
    scores = cross_val_score(clf, X, y, cv=cv)
    print (scores)
    print (("\n平均值: {0:.3f} (均标准差:+/-{1:.3f})").
           format(
               np.mean(scores), # 均值
               sem(scores) )    # 均标准差
           )

def train_and_evaluate(clf, X_test, y_test, train_score):
    
    print ("训练集精度:")   # 训练集精度
    print (train_score)

    print ("\n测试集精度:")    # 测试集精度
    print (clf.score(X_test, y_test))
    
    y_pred = clf.predict(X_test)
    
    print ("\n分类识别报告:")      # 分类识别报告
    print (metrics.classification_report(y_test, y_pred))

    print ("\n混淆矩阵:")           # 混淆矩阵
    print (metrics.confusion_matrix(y_test, y_pred))

### 2.3 被动攻击算法

##### 2.3.1 模型搭建

In [6]:
from sklearn.linear_model import PassiveAggressiveClassifier
clf = PassiveAggressiveClassifier(max_iter=1000, tol=1e-3)

epochs = 20
train_score = 0
batch_size = int(len(train_list) / epochs)

print("开始训练第 1 / 20 批训练")
X, y= get_X_y(train_list[:batch_size], "MachineLearning")
clf.partial_fit(X, y, classes=np.unique(y))
train_score += clf.score(X, y)

for i in range(1,epochs):
    print("开始训练第 %s / 20 批训练" % (i+1))
    X, y = get_X_y(train_list[i*batch_size+1:(i+1)*batch_size], "MachineLearning")
    clf.partial_fit(X, y)
    train_score += clf.score(X, y)

开始训练第 1 / 20 批训练
开始训练第 2 / 20 批训练
开始训练第 3 / 20 批训练
开始训练第 4 / 20 批训练
开始训练第 5 / 20 批训练
开始训练第 6 / 20 批训练
开始训练第 7 / 20 批训练
开始训练第 8 / 20 批训练
开始训练第 9 / 20 批训练
开始训练第 10 / 20 批训练
开始训练第 11 / 20 批训练
开始训练第 12 / 20 批训练
开始训练第 13 / 20 批训练
开始训练第 14 / 20 批训练
开始训练第 15 / 20 批训练
开始训练第 16 / 20 批训练
开始训练第 17 / 20 批训练
开始训练第 18 / 20 批训练
开始训练第 19 / 20 批训练
开始训练第 20 / 20 批训练


##### 2.3.2 模型评估

In [7]:
X_test, y_test = get_X_y(test_list[:2000], "MachineLearning")
train_and_evaluate(clf, X_test, y_test, train_score / epochs)

训练集精度:
0.7484968480156884

测试集精度:
0.15

分类识别报告:
              precision    recall  f1-score   support

           0       1.00      0.01      0.02      1719
           1       0.14      1.00      0.25       281

    accuracy                           0.15      2000
   macro avg       0.57      0.51      0.14      2000
weighted avg       0.88      0.15      0.05      2000


混淆矩阵:
[[  19 1700]
 [   0  281]]


### 2.4 随机梯度下降法SGD

In [8]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(max_iter=1000, tol=1e-3)

epochs = 20
train_score = 0
batch_size = int(len(train_list) / epochs)

print("开始训练第 1 / 20 批训练")
X, y= get_X_y(train_list[:batch_size], "MachineLearning")
clf.partial_fit(X, y, classes=np.unique(y))
train_score += clf.score(X, y)

for i in range(1,epochs):
    print("开始训练第 %s / 20 批训练" % (i+1))
    X, y = get_X_y(train_list[i*batch_size+1:(i+1)*batch_size], "MachineLearning")
    clf.partial_fit(X, y)
    train_score += clf.score(X, y)

开始训练第 1 / 20 批训练
开始训练第 2 / 20 批训练
开始训练第 3 / 20 批训练
开始训练第 4 / 20 批训练
开始训练第 5 / 20 批训练
开始训练第 6 / 20 批训练
开始训练第 7 / 20 批训练
开始训练第 8 / 20 批训练
开始训练第 9 / 20 批训练
开始训练第 10 / 20 批训练
开始训练第 11 / 20 批训练
开始训练第 12 / 20 批训练
开始训练第 13 / 20 批训练
开始训练第 14 / 20 批训练
开始训练第 15 / 20 批训练
开始训练第 16 / 20 批训练
开始训练第 17 / 20 批训练
开始训练第 18 / 20 批训练
开始训练第 19 / 20 批训练
开始训练第 20 / 20 批训练


In [11]:
X_test, y_test = get_X_y(test_list[:2000], "MachineLearning")
train_and_evaluate(clf, X_test, y_test, train_score/20)

训练集精度:
0.7133402962161741

测试集精度:
0.857

分类识别报告:
              precision    recall  f1-score   support

           0       0.86      1.00      0.92      1719
           1       0.14      0.00      0.01       281

    accuracy                           0.86      2000
   macro avg       0.50      0.50      0.46      2000
weighted avg       0.76      0.86      0.79      2000


混淆矩阵:
[[1713    6]
 [ 280    1]]
